In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d

In [ ]:
micro_data = pd.read_pickle('../divestdata/P3/micro/unified_trajectory')
macro_data = pd.read_pickle('../divestdata/P3/aggregate/unified_trajectory')
rep_data = pd.read_pickle('../divestdata/P3/res/unified_trajectory')
print(macro_data.index.names)
print(macro_data.index.levels)
levels=['b_d', 'phi', 'tau', 'eps']
key=[1.25, 0.5, 1.0, 0.01]
micro_data.xs(level=levels, key=key)

In [ ]:

def l_2_distance(d1, d2, key=None, levels=None, axes=None, observable=None):
    
    # get values and times from dataframes
    d1_slice = d1.xs(key=key, level=levels)['mean_trajectory'].unstack('observables')[observable].values.T[0]
    d1_times = d1.xs(key=key, level=levels)['mean_trajectory'].unstack('observables').index.values
    
    d2_slice = d2.xs(key=key, level=levels)['mean_trajectory'].unstack('observables')[observable].values.T[0]
    d2_times = d2.xs(key=key, level=levels)['mean_trajectory'].unstack('observables').index.values
    
    # interpolate to get values on equal timepoints
    d1_ipl = interp1d(d1_times, d1_slice)
    d2_ipl = interp1d(d2_times, d2_slice)
    
    samples = 200
    times = np.linspace(d1_times[0], d1_times[-1], samples)
    d1_data = d1_ipl(times)
    d2_data = d2_ipl(times)
    dd = d1_data - d2_data
    l2 = sum(dd*dd)
    print(l2)
    
    
axes=['phi', 'tau']
levels=['b_d', 'eps']
key=[1.25, 0.05]


l_2_distance(micro_data, macro_data, key=key, levels=levels, observable=['g'])